In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from qiskit import *
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info import Operator
from qiskit import QuantumCircuit
from qiskit.circuit.library import QFT

## General use case using qiskit function 

Note from [qiskit documentation](https://qiskit.org/documentation/stubs/qiskit.aqua.algorithms.Shor.html):

The input integer N to be factored is expected to be odd and greater than 2. Even though this implementation is general, its capability will be limited by the capacity of the simulator/hardware. Another input integer a can also be supplied, which needs to be a co-prime smaller than N.

In [4]:
from qiskit.aqua.algorithms import Shor
from qiskit.aqua import QuantumInstance

In [82]:
lst = [3, 5, 7, 9, 11, 13, 15, 17, 19]

In [85]:
for n in lst:
    
    # Define Shor's algorithm and backend
    shor = Shor(n)
    backend = Aer.get_backend('qasm_simulator')
    
    print()
    print(str(n) + 'qubits case')

    for i in range(5): 

        benchmark_times = []
    
        # Time the simulation runtime and store values
        import time
        start_time = time.time()

        result_dict = shor.run(QuantumInstance(backend, shots=1, skip_qobj_validation=False))
        result = result_dict['factors']

        benchmark_times.append(time.time() - start_time)

        print('factors='+ str(result))
        print(str(n) + ' qubit times = ' + str(benchmark_times))


3qubits case
factors=[]
3 qubit times = [3.8016672134399414]
factors=[]
3 qubit times = [4.010535955429077]
factors=[]
3 qubit times = [4.042552471160889]
factors=[]
3 qubit times = [4.6841301918029785]
factors=[]
3 qubit times = [4.009852886199951]

5qubits case
factors=[]
5 qubit times = [27.72439670562744]
factors=[]
5 qubit times = [26.576203107833862]
factors=[]
5 qubit times = [26.03904366493225]
factors=[]
5 qubit times = [26.353826761245728]
factors=[]
5 qubit times = [26.972954988479614]

7qubits case
factors=[]
7 qubit times = [26.320043325424194]
factors=[]
7 qubit times = [26.718579053878784]
factors=[]
7 qubit times = [25.64811658859253]
factors=[]
7 qubit times = [26.830878496170044]
factors=[]
7 qubit times = [26.270915985107422]

9qubits case
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]

11qubits case
factors=[]
11 qubit times = [112.989268541336

In [88]:
n = 9
    
# Define Shor's algorithm and backend
shor = Shor(n)
backend = Aer.get_backend('qasm_simulator')
    
print()
print(str(n) + 'qubits case')

for i in range(5): 

    benchmark_times = []

    # Time the simulation runtime and store values
    import time
    start_time = time.time()

    result_dict = shor.run(QuantumInstance(backend, shots=1, skip_qobj_validation=False))
    result = result_dict['factors']

    benchmark_times.append(time.time() - start_time)

    print('factors='+ str(result))
    print(str(n) + ' qubit times = ' + str(benchmark_times))


9qubits case
factors=[3]
9 qubit times = [0.0010004043579101562]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]
factors=[3]
9 qubit times = [0.0]


In [6]:
# Define Shor's algorithm and backend
n = 15
shor = Shor(n)
backend = Aer.get_backend('qasm_simulator')
    
print()
print(str(n) + 'qubits case')

for i in range(5): 
    benchmark_times = []
    
    # Time the simulation runtime and store values
    import time
    start_time = time.time()

    result_dict = shor.run(QuantumInstance(backend, shots=1, skip_qobj_validation=False))
    result = result_dict['factors']

    benchmark_times.append(time.time() - start_time)

    print('factors='+ str(result))
    print(str(n) + ' qubit times = ' + str(benchmark_times))


15qubits case
factors=[[3, 5]]
15 qubit times = [130.11084914207458]
factors=[[3, 5]]
15 qubit times = [0.0]
factors=[[3, 5]]
15 qubit times = [0.0]
factors=[[3, 5]]
15 qubit times = [0.0]
factors=[[3, 5]]
15 qubit times = [0.0]


## Specific use case with functions

### Define components of algorithm 

In [ ]:
# N is the number of measurement qubits and m is the number of target qubits for the unitary operator
def initialize_qubits(given_circuit, n, m):
    
    given_circuit.h(range(n))
    given_circuit.x(n+m-1)

In [ ]:
# Define modular exponentiation 

def a_x_mod15(a, x):
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(x):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, x)
    c_U = U.control()
    return c_U

In [ ]:
def modular_exponentiation(given_circuit, n, m, a):
    
    for x in range(n):
        given_circuit.append(a_x_mod15(a, 2**x), [x] + list(range(n, n+m)))         

In [ ]:
def apply_iqft(given_circuit, measurement_qubits):
   
    given_circuit.append(QFT( len(measurement_qubits), do_swaps=False).inverse(), measurement_qubits)

### Construct algorithm using components

In [ ]:
def shor_program(n, m, a):
    
    # set up quantum circuit
    shor = QuantumCircuit(n+m, n)
    
    # initialize the qubits
    initialize_qubits(shor, n, m)
    shor.barrier()

    # apply modular exponentiation
    modular_exponentiation(shor, n, m, a)
    shor.barrier()

    # apply inverse QFT
    apply_iqft(shor, range(n))

    # measure the first n qubits
    shor.measure(range(n), range(n))
    
    return shor
    
n = 4; m = 4; a = 7
mycircuit = shor_program(n, m, a)
mycircuit.draw(output='text')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
counts = execute(mycircuit, backend=simulator, shots=1000).result().get_counts(mycircuit)
from qiskit.visualization import plot_histogram
plot_histogram(counts)

In [ ]:
for measured_value in counts:
    print(f"Measured {int(measured_value[::-1], 2)}")

In [ ]:
# Classical post-processing

from math import gcd

for measured_value in counts:
    measured_value_decimal = int(measured_value[::-1], 2)
    print(f"Measured {measured_value_decimal}")
    
    if measured_value_decimal % 2 != 0:
        print("Failed. Measured value is not an even number")
        continue
    x = int((a ** (measured_value_decimal/2)) % 15)
    if (x + 1) % 15 == 0:
        print("Failed. x + 1 = 0 (mod N) where x = a^(r/2) (mod N)")
        continue
    guesses = gcd(x + 1, 15), gcd(x - 1, 15)
    print(guesses)